# 04-02 Visualisierung-Aufgabe

## Hinweise zur Übung

In dieser Übung wenden Sie die Vorgehensweisen aus der Visualisierungs-Demo in eigenen Abfragen auf die Daten des Kurs-DWH an.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql pandas altair folium

In [ ]:
import os
import sys
import urllib.request
import gzip
import shutil
import pandas as pd
import altair as alt
import folium
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
# Konfiguration
base_url_quellen   = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/08-Datenanalyse+Visualisierung/data"
base_url_reporting = "./"

In [ ]:
# SQlite-Datenbanken aus Github auf den Jupyter-Server herunterladen
urllib.request.urlretrieve(base_url_quellen + "/dwh/reporting.sqlite.gz", base_url_reporting + "reporting.sqlite.gz")

# Die Sqlite-Datenbank ist aufgrund ihrer Größe gezipped und muss vor der Nutzung noch entpackt werden
with gzip.open(base_url_reporting + "reporting.sqlite.gz", "rb") as f_in:
    with open(base_url_reporting + "reporting.sqlite", "wb") as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_reporting      = base_url_reporting + "reporting.sqlite"

db_url_reporting      = "sqlite:///" + db_path_reporting

%sql $db_url_reporting

## Einfache Diagramme mit Altair


### Aufgabe: Stellen Sie die Fallzahlen je Einrichtung für Bielefeld und Mannheim als Balkendiagramm dar

**Erster Schritt:** Erstellen Sie eine SQL-Abfrage aus dem Kurs-DWH, das den Namen des Standorts, den Namen der Einrichtung und die Anzahl der Fälle enthält. Speichern Sie das Ergebnis in einem Dataframe.

**Zweiter Schritt:** Stellen Sie nur die Fallzahlen des Standorts Bielefeld in einem Balkendiagramm dar.

Hinweise:
* filtern Sie den Dataframe auf den Standort Bielefeld
* vergessen Sie nicht, passende Datentyp-Suffixe (z.B. ":Q" im `encoding` anzugeben
* geben Sie passende Titel & Achsenbeschriften an
* Bonusaufgabe: finden Sie heraus, wie Sie die Standorte absteigend nach Fallzahl sortieren können


**Dritter Schritt:** Übernehmen Sie den Code des vorherigen Diagramms und passen ihn so an, dass beide Standorte als Facetten untereinander ausgegeben werden.

Hinweise:
* Altair nutzt defaultmäßig übergreifende Achsen für alle Teilgrafiken eines facettierten Diagramms. Dies führt hier dazu, dass auch beide Teildiagramme alle Einrichtungen beider Standorte ausweisen, obwohl Fallzahlen jeweils nur für den eigentlichen Standort vorliegen. Sie können das abstellen, indem Sie hinter `encode()` die Methode `resolve_scale(y="independent")' aufrufen. Die X-Achse sollte weiterhin einheitlich sein, da die Balken beider Standorte sonst nicht mehr vergleichbar sind.

### Aufgabe: Stellen Sie die Geschlechtsverteilung der ICD10-Diagnosekapitel dar

Diagnosen haben teilweise geschlechtsspezifische Verteilungen (z.B. bei Schwangerschaft/Geburt). In dieser Aufgabe stellen Sie die Geschlechtsverteilungen in Bezug auf die verschiedenen Kapitel des ICD10-Katalogs dar.

**Erster Schritt:** Erstellen Sie eine Abfrage, die für jedes Kapitel der ICD10-Hauptdiagnosen und das Geschlecht die Fallzahl ausweist. Es sollen nur die Hauptdiagnosen der Behandlungsfälle einbezogen werden.

Hinweise:
* Synthea hat zwar den Anspruch, medizinische plausible Datensätze zu generieren, dennoch werden nicht alle Verteilungen tatsächlich realistisch sein.


**Zweiter Schritt:** Erstellen Sie ein facettiertes Kuchendiagramm:

* Kennzahl für die Kreisabschnitte ist die Fallzahl
* Farbe entsprechend des Geschlechts
* Facettierung über die Kapitelnamen des ICD10-Katalogs

### Aufgabe: Stellen Sie die Fallzahl pro Einrichtung und Fallart als gestapeltes Balkendiagramm und Scatterplot dar

In dieser Aufgabe wollen wir die Fallzahlen der verschiedenen Versorgungseinrichtungen in Bezug auf die jeweiligen Fallarten unterscheiden, um beispielsweise zu erkennen, ob einzelne Fallarten nur von bestimmten Typen von Versorgern (z.B. Krankenhäusern) angeboten werden.

**Erster Schritt:** Fragen Sie im Kursdatensatz die Anzahl der Fälle pro Standort, Einrichtung und Fallart ab und speichern das Ergebnis in einem Dataframe `df_fallarten`.


**Zweiter Schritt**: Visualisieren Sie die Fallzahlen in einem gestapelten Balkendiagramm (horizontale Darstellung):
* Setzen Sie die Fallzahl auf die X-Achse
* Setzen Sie den Einrichtungsnamen auf die Y-Achse
* Setzen Sie die Fallart als Farbe ein
* Facettieren Sie das Diagramm nach Standortnamen

**Dritter Schritt**: Visualisieren Sie die Fallzahlen in einem Scatterplot:
* Setzen Sie die Fallart auf die X-Achse
* Setzen Sie den Einrichtungsnamen auf die Y-Achse
* Setzen Sie die Fallzahl als Größe der Punkte ein
* Facettieren Sie das Diagramm nach Standortnamen

## Geovisualisierung mit Folium



### Aufgabe: Stellen Sie die Gesamterlöse der Versorgungseinrichtungen als Geovisualisierung dar

In dieser Aufgabe wollen wir die Erlöse über alle behandelten Fälle summieren und in einer Geovisualisierung als Kreise um die Koordinaten der Versorgungseinrichtungen darstellen.

**Erster Schritt:** Fragen Sie die summierten Erlöse pro Fall in Bezug auf den Namen und die Geokoordinaten der Versorgungseinrichtungen ab und speichern Sie das Ergebnis im Dataframe `df_einrichtungen`.

**Zweiter Schritt**: Visualisieren Sie die Erlöse pro Versorgungseinrichtung auf einer Folium-Karte:
* legen Sie zunächst eine Basiskarte an, die das gesamte Bundesgebiet zeigt
* Platzieren sie pro Einrichtung einen Kreis auf der Karte
* Legen Sie die Größe des Kreises anhand des Erlöses fest (ggf. skalieren, damit die Einrichtungen sich nicht zu stark überlappen)
* stellen Sie die Kreise halbtransparent dar, damit auch überlappende Einrichtungen erkennbar sind
* ergänzen Sie einen Tooltip mit dem Namen der Einrichtung und dem Erlös